In [ ]:
path = ""
from google.colab import drive, files
drive.mount('/content/drive/')
path = '/content/drive/My Drive/App/HPA' #"/content/drive/My Drive/HPA_Validation/"

%tensorflow_version 1.x

In [ ]:
#!pip install mahotas

In [ ]:
import mahotas
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import rescale
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.decomposition import PCA
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature
    
def fd_haralick(image): 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

def fd_hog(image):    
    img = resize(image, (1024, 1024))
    hog_feature = hog(img, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2), 
                                 block_norm='L2-Hys', visualize=False, transform_sqrt=False, 
                                 feature_vector=True, multichannel=True)
    
    
    hog_feature = hog_feature.reshape(hog_feature.shape[0],1)
    scaler = MinMaxScaler()
    data_rescaled = scaler.fit_transform(hog_feature)
    pca = PCA(n_components = 0.95)
    pca.fit(data_rescaled)
    hog_feature_pca = pca.transform(data_rescaled)
    hog_feature_pca = hog_feature_pca.flatten()
    return hog_feature


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D,Lambda
from keras.utils import plot_model
from keras import regularizers
from keras.optimizers import Adam,SGD,Nadam

#from scipy.misc import logsumexp
from scipy.special import logsumexp


def Vgg16_Net():
    # load model without classifier layers
    model = VGG16(include_top=False, input_shape=(1024, 1024, 3))
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(256, activation='relu')(flat1)
    output = Dense(8, activation='sigmoid')(class1)
    model = Model(inputs=model.inputs, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
layer_name = 'dense'
FC_layer_model = Model(inputs=Vgg16_Net.input, outputs=Vgg16_Net.get_layer(layer_name).output)

In [ ]:
### HU, Haralick, HOG and Vgg16 CNN Feature Extraction ###
import os
import cv2
from PIL import Image
from numpy import array
from scipy.sparse import csr_matrix
import scipy
from skimage.transform import resize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

IMG = []
hog_images = []
hog_features = []
global_features = []
features_hog = []
features_haralick = []
features_hu_moments = []
all_features_hand_cnn = []
all_features = []
features_cnn = []
features_hand = []
p_hat = []
batch_features_hand = []
read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
fixed_size = (3000,3000)

PATH = "Image file location" # example: '/content/drive/My Drive/App/HPA/FrenchLab/Image01.jpg'

image = cv2.imread(PATH)
image = cv2.resize(image, fixed_size)
crop_img = image[300:2700, 300:2700]   
       
##############################################
# DCNN Features
###############################################
img = resize(image, (1024,1024))
images = np.array(img) 
FC_output = FC_layer_model.predict(np.expand_dims(images,0))
FC_output = FC_output.reshape(FC_output.shape[1],FC_output.shape[0])

#Data = pd.DataFrame(FC_output)
#Data.to_csv("csv file name") #'/content/drive/My Drive/App/HPA/FrenchLab/Image01_CNN_Features_X.csv') 

####################################
# Handcrafted Feature extraction
####################################
fv_hu_moments = fd_hu_moments(image) 
fv_haralick   = fd_haralick(image)
fv_hog        = fd_hog(image) 

fv_hu_moments= fv_hu_moments.reshape(fv_hu_moments.shape[1],fv_hu_moments.shape[0])
fv_haralick= fv_haralick.reshape(fv_haralick.shape[1],fv_haralick.shape[0])

features_hand = np.concatenate ((fv_hu_moments,fv_haralick,fv_hog), axis=1)
features = np.hstack((features_hand, FC_output)) 

file = PATH + '.csv'
Data = pd.DataFrame(features)
Data.to_csv("file") #'/content/drive/My Drive/App/HPA/FrenchLab/Image01_Features_X.csv') #,index=False)

In [ ]:
def categorical_entropy(proba):
    """Compute the entropy along the last dimension."""
    return - np.sum(proba * np.log(proba + 1), axis=-1)

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def evaluate(classes, y_gt, y_pred):
   
    print("******* Evaluation Score******************")

    y_pred_bin = y_pred #>= threshold_value

    score_f1_macro = f1_score(y_gt, y_pred_bin, average="macro")
    print("Macro f1_socre = {:.6f}".format(score_f1_macro))

    score_f1_micro = f1_score(y_gt, y_pred_bin, average="micro")
    print("Micro f1_socre = {:.6f}".format(score_f1_micro))

    # hamming loss
    h_loss = hamming_loss(y_gt, y_pred_bin)
    print("Hamming Loss = {:.6f}".format(h_loss))

    print('Hamming score: {0}'.format(hamming_score(y_gt, y_pred_bin)))

    # Exact Match ratio:
    print("Exact Match ratio:",np.all(y_pred_bin == y_gt, axis=1).mean())

    mAP = average_precision_score(y_gt, y_pred)
    print("mean Average Precision(mAP) = {:.2f}%".format(mAP * 100))
    ap_classes = []
    for i, cls in enumerate(classes):
        ap_cls = average_precision_score(y_gt[:, i], y_pred[:, i])
        ap_classes.append(ap_cls)
        print("AP({}) = {:.2f}%".format(cls, ap_cls * 100))
    print("mAP = {:.2f}%".format(np.mean(ap_classes) * 100))

    print("Test Sample Size:", y_gt.shape)

    print(multilabel_confusion_matrix(y_gt, y_pred))

def mcc(out_proba,y_test):
    threshold = np.arange(0.01,0.9,0.01)
    acc = []
    accuracies = []
    best_threshold = np.zeros(out_proba.shape[1])
    for i in range(out_proba.shape[1]):
        y_prob = np.array(out_proba[:,i])
        for j in threshold:
            y_pred = [1 if prob>=j else 0 for prob in y_prob]
            acc.append( matthews_corrcoef(y_test[:,i],y_pred))
        acc   = np.array(acc)
        index = np.where(acc==acc.max()) 
        accuracies.append(acc.max()) 
        best_threshold[i] = threshold[index[0][0]]
        acc = []

    print("Best Threshold",best_threshold[:])
    
    y_pred = np.array([[1 if out_proba[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])
    
    total_correctly_predicted = len([i for i in range(len(y_test)) if (y_test[i]==y_pred[i]).sum() == 8])
    print("total_correctly_predicted",total_correctly_predicted)
    
    return y_pred, best_threshold


In [ ]:
"""Multilabel cross validators based on an implementation of the algorithm described in the following paper:
Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of Multi-Label Data. 
Machine Learning and Knowledge Discovery in Databases. ECML PKDD 2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin, Heidelberg.
https://github.com/trent-b/iterative-stratification/blob/master/iterstrat/ml_stratifiers.py
"""

import numpy as np

from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, BaseShuffleSplit, _validate_shuffle_split

class MultilabelStratifiedShuffleSplit(BaseShuffleSplit):
    """Multilabel Stratified ShuffleSplit cross-validator
    Examples
    --------
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)
    >>> for train_index, test_index in msss.split(X, y):
            print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
    ---        
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    TRAIN: [1 2 5 6] TEST: [0 3 4 7]
    """

    def __init__(self, n_splits=10, test_size="default", train_size=None,
                 random_state=None):
        super(MultilabelStratifiedShuffleSplit, self).__init__(
            n_splits, test_size, train_size, random_state)

    def _iter_indices(self, X, y, groups=None):
        n_samples = _num_samples(X)
        y = check_array(y, ensure_2d=False, dtype=None)
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(
                    type_of_target_y))

        n_train, n_test = _validate_shuffle_split(n_samples, self.test_size,
                                                  self.train_size)

        n_samples = y.shape[0]
        rng = check_random_state(self.random_state)
        y_orig = y.copy()

        r = np.array([n_train, n_test]) / (n_train + n_test)

        for _ in range(self.n_splits):
            indices = np.arange(n_samples)
            rng.shuffle(indices)
            y = y_orig[indices]

            test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

            test_idx = test_folds[np.argsort(indices)] == 1
            test = np.where(test_idx)[0]
            train = np.where(~test_idx)[0]

            yield train, test

    def split(self, X, y, groups=None):
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedShuffleSplit, self).split(X, y, groups)


In [ ]:
def DropWeights(x):
    import tensorflow as tf
    p = 0.3
    return tf.nn.dropout(x, rate = p) * (1 - p)

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=100)
mc = ModelCheckpoint('/content/drive/My Drive/App/HPA/Best_HNetModel.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=2, min_lr=0.000001, verbose=1, mode="auto", cooldown=0)

def main():
  
    kf = MultilabelStratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

    for train_index, test_index in kf.split(X, Y):    
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        model = keras.Sequential()
        model.add(Dense(2048, input_shape=(3996,),kernel_initializer="he_uniform", activation='relu'))
        model.add(Dense(2048, kernel_initializer="he_uniform", activation='relu'))
        model.add(Dense(1024, kernel_initializer="he_uniform", activation='relu'))
        model.add(Dense(512,  kernel_initializer="he_uniform", activation='relu'))
        model.add(Lambda(DropWeights))
        model.add(Dense(8, activation='sigmoid'))        
        adam = Adam(lr= 0.000001, amsgrad=True)

        model.compile(adam, 'binary_crossentropy', metrics=[f1_m, 'accuracy'])
        
        model.fit(X_train, Y_train, epochs= 1000, batch_size=32, verbose=0, callbacks=[es, mc, reduce_lr])     

        from keras.models import load_model
        model = load_model ("/content/drive/My Drive/App/HPA/Best_HNetModel.h5")  

        num_samples = 1000
        ens_preds = []
        for seed in range(num_samples):
            ens_preds.append(model.predict(X_test))
        Yt_hat = np.array(ens_preds)

        prediction = np.mean(Yt_hat, axis = 0)
        Y_predict = mcc(prediction,Y_test)

        evaluate(classes, Y_test, Y_predict)

main()

In [ ]:
class_names = ['Leydig Cells', 'Elongated/late Spermatids Cells', 'Pachytene Spermatocytes Cells', 'Peritubular Cells', 
           'Preleptotene spermatocytes Cells', 'Round/early Spermatids Cells', 'Sertoli Cells', 'Spermatogonia Cells']
classes = class_names
cell_type = 0 # 0: Leydig, .. 7: Spermatogonia
T = 1000

for t in range(T):
    p_hat.append(model.predict(X_test,verbose=0))
p_hat = np.array(p_hat)


y_gt = Y_test[:,cell_type:cell_type+1]
y_pred = y_predict[:,cell_type:cell_type+1]

tn, fp, fn, tp = confusion_matrix(y_gt.flatten(),y_pred.flatten()).ravel()
 
# Plug-in estimate of Entropy    
proba_avg = p_hat.mean(axis=0)
entropy_expected = categorical_entropy(proba_avg)

# Jackknife correction for Bias
proba_loo = proba_avg + (proba_avg - p_hat) / (len(p_hat) - 1)
expected_entropy_p = categorical_entropy(proba_loo).mean(axis=0)
entropy_expected += (len(p_hat) - 1) * (entropy_expected - expected_entropy_p)

MCDW_samples = entropy_expected - categorical_entropy(p_hat).mean(axis=0)
MCDW_samples = np.array(MCDW_samples)

#uncertainty_norm = (MCDW_samples - np.min(MCDW_samples)) / (np.max(MCDW_samples) - np.min(MCDW_samples))

# For Multi-Class Classification
#proba_samples_sorted = np.sort(np.mean(MCDW_samples, axis = 0))[::-1]
#CPPD = proba_samples_sorted[:,-1] - proba_samples_sorted[:, -2]

# For Multi-Label Classification
CPPD = (proba_avg[:,0:1])
CPPD = 2*CPPD.flatten() -1
GTL = CPPD.flatten()/MCDW_samples.flatten()

GTL_samples =  (GTL - np.min(GTL)) / (np.max(GTL) - np.min(GTL))
GTL_samples = np.array(GTL_samples)